In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('DATASETS/Project_dataset.csv')

In [2]:
data.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,artist_and_song,genre,Lyrics
0,0.03100,0.610,200400,0.926,0.001200,0.000000,0.0821,-4.843,0.0,0.0479,172.638,0.0,0.861,*nsync /// bye bye bye,"['Dance', 'Pop']","(Hey, Hey)\nBye, Bye, Bye\nBye, Bye\nBye, Bye\..."
1,0.08660,0.748,253600,0.811,0.000015,0.083333,0.0282,-5.213,0.0,0.0907,93.963,0.0,0.862,*nsync /// girlfriend,['Dance'],Would you be my girlfriend?\nWould you be my g...
2,0.43000,0.704,292000,0.409,0.000000,0.166667,0.1090,-8.581,0.0,0.0597,113.863,0.0,0.495,*nsync /// gone,['Dance'],"There's a thousand words that I could say,\nTo..."
3,0.00993,0.810,206333,0.857,0.000011,0.250000,0.0528,-4.658,0.0,0.0881,104.502,0.0,0.646,*nsync /// it makes me ill,['Dance'],Justin:\nI was hanging with the fellas\nSaw yo...
4,0.04070,0.636,192427,0.873,0.000001,0.083333,0.0372,-4.672,0.0,0.0710,165.071,0.0,0.908,*nsync /// it's gonna be me,"['Dance', 'Pop']",You might be hurt babe\nThat ain't no lie\nYou...


### Audio Baseline